# T1 Mapping - IR GRE

T1 mapping using a spoiled gradient echo sequence where a single readout line is acquired after an inversion pulse. 
Multiple images are obtained at different inversion times.

### Imports

In [ ]:
from pathlib import Path

import ismrmrd
import matplotlib.pyplot as plt
import MRzeroCore as mr0
import numpy as np
import torch
from einops import rearrange
from mrpro.algorithms.reconstruction import DirectReconstruction
from mrpro.data import KData
from mrpro.data.traj_calculators import KTrajectoryCartesian
from mrpro.data.traj_calculators import KTrajectoryIsmrmrd
from mrpro.operators import DictionaryMatchOp
from mrpro.operators.models import InversionRecovery

from mrseq.scripts.t1_inv_rec_gre_single_line import main as create_seq
from mrseq.utils import sys_defaults

### Settings

In [ ]:
image_matrix_size = [128, 128]
inversion_times = [0.025, 0.050, 0.3, 0.6, 1.2, 2.4, 4.8, 8.0]
fname_seq = Path('/Users/kolbit01/Desktop/mp/t1_inv.seq')
fname_mrd = Path('/Users/kolbit01/Desktop/mp/t1_inv.mrd')

### Create digital phantom

In [ ]:
phantom = mr0.util.load_phantom(image_matrix_size)

### Create sequence

In [ ]:
sequence = create_seq(
    system=sys_defaults,
    show_plots=False,
    fov_xy=float(phantom.size.numpy()[0]),
    tr=20,
    n_readout=image_matrix_size[0],
    n_phase_encoding=image_matrix_size[1],
    inversion_times=inversion_times,
)
sequence.write(str(fname_seq))

### Simulate sequence

In [ ]:
mr0_sequence = mr0.Sequence.import_file(str(fname_seq))
signal, ktraj_adc = mr0.util.simulate(mr0_sequence, phantom, accuracy=1e-5)
mr0.sig_to_mrd(fname_mrd, signal, sequence, verbose=1)

In [ ]:
with ismrmrd.File(fname_mrd, 'r') as file:
    ds = file[list(file.keys())[-1]]
    ismrmrd_header = ds.header
    acquisitions = ds.acquisitions[:]

# ismrmrd_header.encoding[0].encodedSpace.matrixSize.x = 128
# ismrmrd_header.encoding[0].encodedSpace.matrixSize.y = 128
# ismrmrd_header.encoding[0].encodedSpace.matrixSize.z = 1
# ismrmrd_header.encoding[0].reconSpace.matrixSize.x = 128
# ismrmrd_header.encoding[0].reconSpace.matrixSize.y = 128
# ismrmrd_header.encoding[0].reconSpace.matrixSize.z = 1

ismrmrd_header.sequenceParameters.TI = [ti * 1000 for ti in ismrmrd_header.sequenceParameters.TI]

exp = ismrmrd.xsd.experimentalConditionsType()
exp.H1resonanceFrequency_Hz = 127729200  # 3T
ismrmrd_header.experimentalConditions = exp

# Create new file
if fname_mrd.exists():
    fname_mrd.unlink()
ds = ismrmrd.Dataset(fname_mrd)
ds.write_xml_header(ismrmrd_header.toXML())

for acq in acquisitions:
    acq.read_dir[0] = 1
    acq.phase_dir[1] = 1
    acq.slice_dir[2] = 1
    ds.append_acquisition(acq)
ds.close()

### Reconstruct images

In [ ]:
kdata = KData.from_file(fname_mrd, trajectory=KTrajectoryIsmrmrd(normalize=True))
kdata = KData.from_file(fname_mrd, trajectory=KTrajectoryCartesian())
kdata.header.encoding_matrix.x = 128
kdata.header.encoding_matrix.y = 128
kdata.header.encoding_matrix.z = 1
kdata.header.recon_matrix.x = 128
kdata.header.recon_matrix.y = 128
kdata.header.recon_matrix.z = 1
kdata.traj.kz[:] = 0
recon = DirectReconstruction(kdata, csm=None)
idata = recon(kdata)

In [ ]:
idat = idata.data.abs().numpy().squeeze()
fig, ax = plt.subplots(1, idat.shape[0], figsize=(15, 5))
for i in range(idat.shape[0]):
    ax[i].imshow(idat[i, :, :], cmap='gray')

### Estimate T1 maps

In [ ]:
dictionary = DictionaryMatchOp(InversionRecovery(ti=idata.header.ti), index_of_scaling_parameter=0)
dictionary.append(torch.tensor(1.0), torch.linspace(0.1, 5.0, 1000)[None, :])
m0_match, t1_match = dictionary(idata.data[:, 0, 0])
m0_match /= m0_match.abs().max()

t1_input = rearrange(phantom.T1.numpy().squeeze()[::-1, ::-1], 'x y -> y x')
obj_mask = np.zeros_like(phantom.PD)
obj_mask[phantom.PD > 0] = 1
obj_mask = rearrange(np.squeeze(obj_mask)[::-1, ::-1], 'x y -> y x')
t1_output = t1_match.numpy().squeeze() * obj_mask
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(t1_input, vmin=0, vmax=3)
ax[1].imshow(t1_output, vmin=0, vmax=3)
ax[2].imshow(np.abs(t1_input - t1_output), vmin=0, vmax=3)